In [1]:
import numpy as np
import multiprocessing

In [2]:
invalid_chromosome = set()
Pc = 0.9
Pm = 0.05
population_size = 100
chromosome_size = 100
resources = 20

In [3]:
# Generate a random population of chromosomes
population = np.random.randint(0, resources, size=(population_size, chromosome_size))

# Generate random deadlines for each task in the chromosome
deadlines = np.random.randint(1,resources , chromosome_size)

print("Random Population:")
print(population)

print("\nRandom Deadlines:")
print(deadlines)


Random Population:
[[ 7 17 18 ... 18  4  5]
 [11 15 18 ... 19 10 18]
 [11 19  1 ... 12 15 10]
 ...
 [15  4  3 ... 11 16 15]
 [ 9  3  6 ...  3  8 17]
 [14  2  0 ...  4  7 17]]

Random Deadlines:
[ 9  8 14  9 10  1  8  1  4  7  8 16 19 12  7  5  3  5  6 16 13 18  4 16
 16  7  8  7 18 10 11 15  7  8 19 14 18 11 13  9 14 10  1  1 19  8  5  1
 15  4  9  1  5 19  3 18 15  5  6 17 15 16 18  1 11 14  3  7 18 19  9 16
 19 15 11  3 19 18  2 17  1 10  9 19 15  7 16  6 14  4 15 17  6 14 13  9
  6 15  9 13]


In [4]:
"""
    Schedule tasks based on the given chromosome and the number of resources.

    Parameters:
    - chromosome: List or array representing the task-resource assignments.
    - num_resources: Number of available resources.

    Returns:
    - completion_times: List of completion times for each task.
"""
def schedule(chromosome, num_resources):
    current_time = 0
    completion_times = [0] * len(chromosome)
    visited = [False] * len(chromosome)
    resource_load = [0] * resources
    for i in range( 0 , len(chromosome)):    
        current_time += 1
        resource_availability = [True] * num_resources
        for j in range(0 , len(chromosome)):
            if resource_availability[chromosome[j]] and not visited[j] :
                completion_times[j] = current_time + 1
                resource_availability[chromosome[j]] = False
                visited[j] = True
                resource_load[chromosome[j]] += 1

    return completion_times , resource_load

# chromosome = np.random.randint(0, resources , chromosome_size )
# completion_time = schedule(chromosome , resources)
# print(completion_time)
# print(resource_load)



In [5]:
"""
    Calculate the fitness function based on SLA, missed tasks, and standard deviation.

    Parameters:
    - completion_times: Array of completion times for each task.
    - deadlines: Array of deadlines for each task.
    - resource_loads: List or array of resource loads.
    - alpha: Coefficient for SLA.
    - beta: Coefficient for missed tasks.
    - gamma: Coefficient for standard deviation.

    Returns:
    - fitness: Fitness value.
"""

def calculate_fitness(completion_times, deadlines, resource_loads, alpha=0.25, beta=0.25, gamma=0.5):
    sla = np.sum(np.maximum(completion_times - deadlines, 0))
    missed_tasks = np.sum(completion_times > deadlines)
    sd = np.std(resource_loads)
    # Fitness function
    fitness = (alpha * sla) * (beta * missed_tasks) * (gamma * sd)  # Adjust 1.0 based on the scale of your fitness
    return fitness
# fitness = calculate_fitness(completion_time, deadlines,resource_load )
# print(f"Fitness of the chromosome : {fitness}")

In [6]:
def crossover(parent1, parent2, PC):
    """
    Perform crossover operation on two parent chromosomes.

    Parameters:
    - parent1: First parent chromosome
    - parent2: Second parent chromosome
    - PC: Probability of crossover (0 to 1)

    Returns:
    - offspring1: First offspring chromosome
    - offspring2: Second offspring chromosome
    """
    # Generate random numbers for crossover
    r1 = np.random.rand()
    r2 = np.random.rand()

    # Check if crossover should be performed based on the probability and random numbers
    if r1 < PC:
        i = np.random.randint(0, len(parent1))
        if r2 < PC:
            j = np.random.randint(0, len(parent1))
            crossover_point = min(i, j)
            offspring1 = np.concatenate((parent1[:crossover_point], parent2[crossover_point:]))
            offspring2 = np.concatenate((parent2[:crossover_point], parent1[crossover_point:]))
        else:
            offspring1 = parent1.copy()
            offspring2 = parent2.copy()
    else:
        # If crossover is not performed, return the original parents
        offspring1 = parent1.copy()
        offspring2 = parent2.copy()

    return offspring1, offspring2

In [7]:
def mutation(chromosome, Pm):
    """
    Apply mutation to a chromosome with a certain probability.

    Parameters:
    - chromosome: The chromosome to be mutated.
    - pm: Probability of mutation.

    Returns:
    - mutated_chromosome: The mutated chromosome.
    """
    original_chromosome = chromosome
    choice = np.random.uniform()
    if choice < Pm :
        mp1 = np.random.randint(0, len(chromosome))
        mp2 = np.random.randint(0, len(chromosome))

        # Swap two rows (genes) of the chromosome
        chromosome[mp1], chromosome[mp2] = chromosome[mp2], chromosome[mp1]

        # Check if the new chromosome satisfies the problem
        if tuple(chromosome) not in invalid_chromosome:
            return chromosome
        else:
            # If not valid, add to invalid chromosomes
            invalid_chromosome.add(tuple(chromosome.copy()))
            return original_chromosome
    else:
        return chromosome


In [8]:

def roulette_wheel_selection(chromosomes, fitness_values):
    """
    Perform roulette wheel selection to choose two parents based on fitness values.

    Parameters:
    - chromosomes: List of chromosomes.
    - fitness_values: List of fitness values corresponding to each chromosome.

    Returns:
    - parent1, parent2: Two selected parents.
    """
    total_fitness = np.sum(fitness_values)

    # Calculate probabilities for each chromosome based on their fitness
    probabilities = fitness_values / total_fitness

    # Perform roulette wheel selection to choose the first parent
    parent1_index = np.random.choice(len(chromosomes), p=probabilities)
    parent1 = chromosomes[parent1_index]

    # Exclude the first parent's index from the selection for the second parent
    remaining_indices = [i for i in range(len(chromosomes)) if i != parent1_index]

    # Recalculate probabilities for the remaining chromosomes
    remaining_probabilities = probabilities[remaining_indices] / np.sum(probabilities[remaining_indices])

    # Perform roulette wheel selection to choose the second parent from the remaining chromosomes
    parent2_index = np.random.choice(remaining_indices, p=remaining_probabilities)
    parent2 = chromosomes[parent2_index]

    return parent1, parent2

In [9]:
#calculating fitness of current population


In [10]:
def selection(chromosomes, fitness_values):
    """
    Perform roulette wheel selection to choose two parents based on the highest fitness values.

    Parameters:
    - chromosomes: List of chromosomes.
    - fitness_values: List of fitness values corresponding to each chromosome.

    Returns:
    - parent1, parent2: The two chromosomes with the highest probabilities.
    """
    total_fitness = np.sum(fitness_values)

    # Check if total_fitness is 0, meaning all fitness values are 0
    if total_fitness == 0:
        # If all fitness values are 0, select two random chromosomes
        return np.random.choice(chromosomes, size=2, replace=False)

    # Calculate probabilities for each chromosome based on their fitness
    probabilities = fitness_values / total_fitness

    # Choose the indices of the top two chromosomes with the highest probabilities
    top_two_indices = np.argsort(probabilities)[-2:]

    # Get the top two chromosomes
    parent1, parent2 = chromosomes[top_two_indices[0]], chromosomes[top_two_indices[1]]

    return parent1, parent2
# par1 , par2 = selection(population, fitness_value)
# child1, child2 = crossover(par1, par2, Pc)
# child1 = mutation(child1, Pm)
# child2 = mutation(child2, Pm)


In [11]:
def selection_invalid(chromosomes, fitness_values):
    """
    Perform roulette wheel selection to choose two parents based on the lowest fitness values.

    Parameters:
    - chromosomes: List of chromosomes.
    - fitness_values: List of fitness values corresponding to each chromosome.

    Returns:
    - parent1, parent2: The two chromosomes with the lowest probabilities.
    """
    total_fitness = np.sum(fitness_values)

    # Check if total_fitness is 0, meaning all fitness values are 0
    if total_fitness == 0:
        # If all fitness values are 0, select two random chromosomes
        return np.random.choice(chromosomes, size=2, replace=False)

    # Calculate probabilities for each chromosome based on their fitness
    probabilities = fitness_values / total_fitness

    # Choose the indices of the bottom two chromosomes with the lowest probabilities
    bottom_two_indices = np.argsort(probabilities)[:2]
    return bottom_two_indices
    # Get the bottom two chromosomes
    # parent1, parent2 = chromosomes[bottom_two_indices[0]], chromosomes[bottom_two_indices[1]]

    # return parent1, parent2
# bottom_two_indices = selection_invalid(population, fitness_value)
# print(par1)
# print(population[bottom_two_indices[1]])

In [12]:
def calculate_fitness_for_population(population):
    fitness_value = [0] * population_size
    for i , chromosome in enumerate(population):
        completion_time , resource_load = schedule(chromosome, resources)
        fitness_value[i] = calculate_fitness(completion_time,deadlines,resource_load)
    return fitness_value


In [13]:
Generation = 0
no_improvement_threshold = 10
best_fitness = float('inf')  
no_improvement_count = 0
while Generation < 1000 :
    fitness_values = calculate_fitness_for_population(population)    
    # best_chromosome_index = np.argmin(fitness_values)
    # current_best_fitness = fitness_values[best_chromosome_index]
    # print(best_chromosome_index)
    # ## Teemination condition ##
    # if current_best_fitness < best_fitness:
    #     best_fitness = current_best_fitness
    #     no_improvement_count = 0  
    # else:
    #     no_improvement_count += 1
    # if no_improvement_count >= no_improvement_threshold:
    #     print(f"Terminating due to no improvement for {no_improvement_threshold} iterations.")
    #     break
    ## Teemination condition ##
    


    ##Selection of fittest parents
    par1, par2 = selection(population, fitness_values)

    ##Performing the crossover on fittest
    child1, child2 = crossover(par1, par2, Pc)

    ##Performing the mutation on child
    child1 = mutation(child1, Pm)
    child2 = mutation(child2, Pm)

    ##Finding two worst 
    bottom_two_indices = selection_invalid(population, fitness_values)

    ##inserting two worst in invaid set
    invalid_chromosome.add(tuple(population[bottom_two_indices[0]]))
    invalid_chromosome.add(tuple(population[bottom_two_indices[1]]))
    population[bottom_two_indices[0]] = child1
    population[bottom_two_indices[1]] = child2

    Generation += 1


# print(Generation)

In [14]:
fitness_value_cur = calculate_fitness_for_population(population)
best_chromosome_index = np.argmax(fitness_value_cur)
best_schedule = population[best_chromosome_index]
completion_time , resource_load = schedule(best_schedule,resources)
print(f"Best schedule out of all : {best_schedule}")
print(f"Completion times : {completion_time}")
print(f"resources load : {resource_load}")

Best schedule out of all : [ 5  0  8 14 17  8 11  5  3 19 19 17  6 16 12 11  1  7  8 19  4 19 16  0
  0  5 11 19  3  4 14 16  7  3  5  6 12  0  4  9  8  1 19  0  4 11 15 19
 18  5 11  4 19 14  5  5  5 14  0 15  2  3 11  5 10  0  5 11 16 11  8  6
 18 16  5  0 12  9  4 14  3 11 12  8  3  5  9  2  8  3  7  7  5 17 18 19
 19 12  5  5]
Completion times : [2, 2, 2, 2, 2, 3, 2, 3, 2, 2, 3, 3, 2, 2, 2, 3, 2, 2, 4, 4, 2, 5, 3, 3, 4, 4, 4, 6, 3, 3, 3, 4, 3, 4, 5, 3, 3, 5, 4, 2, 5, 3, 7, 6, 5, 5, 2, 8, 2, 6, 6, 6, 9, 4, 7, 8, 9, 5, 7, 3, 2, 5, 7, 10, 2, 8, 11, 8, 5, 9, 6, 4, 3, 6, 12, 9, 4, 3, 7, 6, 6, 10, 5, 7, 7, 13, 4, 3, 8, 8, 4, 5, 14, 4, 4, 10, 11, 6, 15, 16]
resources load : [8, 2, 2, 7, 6, 15, 3, 4, 7, 3, 1, 9, 5, 0, 5, 2, 5, 3, 3, 10]
